In [76]:
from lxml import etree
from itertools import islice
import wikitextparser as wtp

In [175]:
DATA_PATH = 'data/ukwiki-latest-pages-articles1.xml'
# DATA_PATH = 'data/test.xml'
SKIP = 1600
TAKE = 100

In [177]:
def _ext_ns(tag, ns):
    if ns:
        return f'{{{ns}}}{tag}'
    else:
        return ns

def _get_child(elem, tag):
    ns = elem.nsmap.get(None)
    return elem.find(_ext_ns(tag, ns))

def _filter_page(page):
    ns = page.nsmap.get(None)
    has_redirect = _ext_ns('redirect', ns) in (child.tag for child in page)
    return not has_redirect

def _filter_page_text(text, log = False):
    parsed = wtp.parse(text)
    target_templates = ('особа', )
    target_args = ('народженн', 'смерті')
    for t in parsed.templates:
        t_name = t.normal_name().lower().strip()
        if t_name in target_templates:
            if log:
                print(f'matched template "{t_name}" with one of {target_templates}')
            return True
        for t_arg in t.arguments:
            t_arg_name = t_arg.name.lower().strip()
            if t_arg_name in target_args:
                if log:
                    print(f'matched template arg "{t_arg_name}" with one of {target_args}')
                return True
            for a_target in target_args:
                if a_target in t_arg_name:
                    if log:
                        print(f'matched template arg "{t_arg_name}" with "{a_target}"')
                    return True
    
    target_sections = ('біографі', 'життєпис')
    for s in parsed.sections:
        if not s.title:
            continue
        s_title = s.title.lower().strip()
        if s_title in target_sections:
            if log:
                print(f'matched section "{s_title}" with one of "{target_sections}"')
            return True
        for s_target in target_sections:
            if s_target in s_title:
                if log:
                    print(f'matched section "{s_title}" with "{s_target}"')
                return True
    return False

context = etree.iterparse(DATA_PATH, tag='{*}page')

for _, page_elem in islice(context, SKIP, SKIP + TAKE):
    ns = page_elem.nsmap.get(None)
    # filter redirects
    if not _filter_page(page_elem):
        page_elem.clear()
        continue
    revis = _get_child(page_elem, 'revision')
    assert revis is not None
    page_title_elem = _get_child(page_elem, 'title')
    page_text_elem = _get_child(revis, 'text')
    assert page_text_elem is not None
    page_title = page_title_elem.text
    page_text = page_text_elem.text
    if not _filter_page_text(page_text):
        print(f'Skipped "{page_title}"')
        page_elem.clear()
        continue
    print(f'Accepted "{page_title}"')
    page_elem.clear()
    

Skipped "191"
Skipped "192"
Skipped "193"
Skipped "194"
Skipped "195"
Skipped "196"
Skipped "197"
Skipped "198"
Skipped "199"
Skipped "200"
Accepted "Донченко Олесь Васильович"
Accepted "Грабовський Павло Арсенович"
Accepted "Панас Мирний"
Accepted "Гулак-Артемовський Петро Петрович"
Accepted "Іваничук Роман Іванович"
Accepted "Андріяшик Роман Васильович"
Accepted "Микитенко Іван Кіндратович"
Accepted "Васильченко Степан Васильович"
Accepted "Тодось Осьмачка"
Accepted "Мушкетик Юрій Михайлович"
Accepted "Яновський Юрій Іванович"
Skipped "Українська діаспора"
Accepted "Леопольд фон Захер-Мазох"
Skipped "Вінниця"
Skipped "Львів"
Skipped "Івано-Франківськ"
Accepted "Беніто Муссоліні"
Accepted "Марк Антоній"
Accepted "Жан-Оноре Фраґонар"
Accepted "Карл VI Габсбург"
Skipped "201"
Skipped "202"
Skipped "203"
Skipped "204"
Skipped "205"
Skipped "206"
Skipped "207"
Skipped "208"
Skipped "209"
Skipped "210"
Skipped "211"
Skipped "212"
Skipped "213"
Skipped "214"
Skipped "215"
Skipped "216"
Skip

: 

In [2]:
import mwparserfromhell as mwp
import wikitextparser as wtp
import re

In [3]:
text = """
{{Інші значення|тип=прізвище|Мороз}}
{{Картка:Лідер
 | ім'я =Олександр Олександрович Мороз
 | зображення = Moroz Yushchenko cropped.jpg
 | розмір_зображення = 200px
 | підпис_зображення =
| посада =  7-й [[Голова Верховної Ради України]]
| початок_терміну = [[6 липня]] [[2006]]
| кінець_терміну =  [[23 листопада]] [[2007]]
| попередник = [[Литвин Володимир Михайлович|Володимир Литвин]]
| наступник = [[Яценюк Арсеній Петрович|Арсеній Яценюк]]
| президент = [[Ющенко Віктор Андрійович|Віктор Ющенко]]
| посада2 =  3-й [[Список Голів Верховної Ради України|Голова Верховної Ради України]]
| початок_терміну2 = [[18 травня]] [[1994]]
| кінець_терміну2 = [[12 травня]] [[1998]]
| попередник2 = [[Плющ Іван Степанович|Іван Плющ]]
| наступник2 = [[Ткаченко Олександр Миколайович (політик)|Олександр Ткаченко]]
| президент2 = [[Кравчук Леонід Макарович|Леонід Кравчук]] &lt;br/&gt; [[Кучма Леонід Данилович|Леонід Кучма]]
 &lt;!--| дата_народження = 29.2.1944&lt;ref name=&quot;ЕІУдата&quot; /&gt;[http://resource.history.org.ua/cgi-bin/eiu/history.exe?&amp;I21DBN=EIU&amp;P21DBN=EIU&amp;S21STN=1&amp;S21REF=10&amp;S21FMT=eiu_all&amp;C21COM=S&amp;S21CNR=20&amp;S21P01=0&amp;S21P02=0&amp;S21P03=TRN=&amp;S21COLORTERMS=0&amp;S21STR=Moroz_O Мороз Олександр Олександрович], Енциклопедія історії України&lt;/ref&gt;
 | місце_народження = [[Буда (Таращанський район)]],&lt;br/&gt;[[Київська область]],&lt;br/&gt;[[Українська Радянська Соціалістична Республіка|Українська РСР]], [[Союз Радянських Соціалістичних Республік|СРСР]]--&gt;
 | дата_смерті = 
 | місце_смерті =
 | батько = Олександр Леонтійович
 | мати = Ганна Євдокимівна
 | дружина = Валентина Андріївна
 | діти = дочки Ірина та Руслана
 | партія = [[Соціалістична партія України]]
 | рід_діяльності = [[політик]]
 | підпис = Oleksandr Moroz Signature 1996.png
 | нагороди =
 {{(!}} style=&quot;background: transparent&quot;
 {{!}} {{Медаль «За трудову доблесть»}}
 {{!}} {{Золотий письменник України}}
 {{!)}}
 }}
{{Депутат
| депутат1    = {{прапорець20|UKR}} [[Народний депутат України]]
| скликання1 = [[народні депутати України 1-го скликання|1-го скликання]]
| партія1 = ''з 1991'' [[Соціалістична партія України]] (група ''Аграрники'')
| початок1 = [[15 травня]] [[1990]]
| кінець1 = [[10 травня]] [[1994]]
| скликання2 = [[народні депутати України 2-го скликання|2-го скликання]]
| партія2 = [[Соціалістична партія України]]
| початок2 = [[12 травня]] [[1994]]
| кінець2 = [[12 травня]] [[1998]]
| скликання3 = [[народні депутати України 3-го скликання|3-го скликання]]
| партія3 = [[Соціалістична партія України]]
| початок3 = [[12 травня]] [[1998]]
| кінець3 = [[14 травня]] [[2002]]
| скликання4 = [[народні депутати України 4-го скликання|4-го скликання]]
| партія4 = [[Соціалістична партія України]]
| початок4 = [[14 травня]] [[2002]]
| кінець4 = [[25 травня]] [[2006]]
| скликання5 = [[народні депутати України 5-го скликання|5-го скликання]]
| партія5 = [[Соціалістична партія України]]
| початок5 = [[25 травня]] [[2006]]
| кінець5 = [[23 листопада]] [[2007]]
}}

'''Моро́з Олекса́ндр Олекса́ндрович''' ({{н.}}&amp;nbsp;{{ДН|29|02|1944}}, с.&amp;nbsp;[[Буда (Таращанський район)|Буда]], [[Таращанський район]], [[Київська область]], [[Українська Радянська Соціалістична Республіка|Українська РСР]], [[Союз Радянських Соціалістичних Республік|СРСР]])&amp;nbsp;— [[Україна|український]] [[політик]] та [[громадський діяч]], двічі [[Голова Верховної Ради України]] у 1994—1998 та 2006—2007 роках. Лідер колишньої [[Соціалістична партія України|Соціалістичної партії України]]. Кандидат у [[Президент України|президенти України]] на виборах [[Вибори Президента України 1994|1994]], [[Вибори Президента України 1999|1999]], [[Вибори Президента України 2004|2004]], [[Вибори Президента України 2010|2010]] та [[Вибори Президента України 2019|2019]].

== Життєпис ==
Насправді народився [[19 лютого]] [[1944]] року в селі [[Буда (Таращанський район)|Буда]] [[Таращанський район|Таращанського району]] [[Київська область|Київської області]]. Через [[Німецько-радянська війна|Німецько-радянську війну]], що тривала тоді, дитину зареєстрували лише [[29 лютого]],&lt;ref name=&quot;ЕІУдата&quot;&gt;[http://resource.history.org.ua/cgi-bin/eiu/history.exe?&amp;I21DBN=EIU&amp;P21DBN=EIU&amp;S21STN=1&amp;S21REF=10&amp;S21FMT=eiu_all&amp;C21COM=S&amp;S21CNR=20&amp;S21P01=0&amp;S21P02=0&amp;S21P03=TRN=&amp;S21COLORTERMS=0&amp;S21STR=Moroz_O Мороз Олександр Олександрович] {{Webarchive|url=https://web.archive.org/web/20200830130509/http://resource.history.org.ua/cgi-bin/eiu/history.exe?&amp;I21DBN=EIU&amp;P21DBN=EIU&amp;S21STN=1&amp;S21REF=10&amp;S21FMT=eiu_all&amp;C21COM=S&amp;S21CNR=20&amp;S21P01=0&amp;S21P02=0&amp;S21P03=TRN=&amp;S21COLORTERMS=0&amp;S21STR=Moroz_O |date=30 серпня 2020 }}, Енциклопедія історії України&lt;/ref&gt; і власне цей день вважається офіційною датою народження&lt;ref name=&quot;segodnya&quot;&gt;[http://www.segodnya.ua/news/908210.html Газета «Сегодня»: Мороз празднует два дня рождения, а Сабо&amp;nbsp;— раз в четыре года] {{Webarchive|url=https://web.archive.org/web/20080312180129/http://www.segodnya.ua/news/908210.html |date=12 березня 2008 }}, Сегодня, 28 февраля 2008{{Ref-ru}}&lt;/ref&gt;.

У 1965 р. закінчив [[Національний університет біоресурсів і природокористування України|Українську сільськогосподарську академію]] за фахом «[[інженер-механік]]». У 1983 р.&amp;nbsp;— [[Вища партійна школа (Київ)|Вищу партійну школу]] при [[Центральний комітет Комуністичної партії України|ЦК Компартії України]], отримав диплом [[політолог]]а.

У 1966—1974 рр.&amp;nbsp;— викладач, завідувач відділу механізації сільського господарства Таращанського технікуму механізації. У 1974—1975 рр.&amp;nbsp;— старший [[інженер-технолог]], старший виконроб мехзагону Таращанського райоб'єднання «Сільгосптехніка». У 1975—1976 рр.&amp;nbsp;— старший інженер Київського обласного об'єднання «Сільгосптехніка».

Від 1976 року&amp;nbsp;— завсектору, заступник завідувача сільськогосподарського відділу Київського обкому [[Комуністична партія України (1918—1991)|КПУ]]. У 1983—1989 рр.&amp;nbsp;— секретар Київської облпрофради, перший секретар РК КПУ. Від 1989 року&amp;nbsp;— завідувач аграрного відділу обкому КПУ.

== Політична діяльність ==
У 1990 р. обраний [[Народний депутат України|народним депутатом України]]. Увійшов до комісії з питань АПК та групи «Аграрники». Став лідером комуністичної більшості (так звана «[[Група 239]]»). Після заборони КПУ був одним із ініціаторів створення [[Соціалістична партія України|Соціалістичної партії України]] і очолив її.

У березні 1994 р. знову обраний народним депутатом, у травні того ж року&amp;nbsp;— головою Верховної Ради України&lt;ref&gt;{{Постанова ВРУ|14/94-ВР|18 травня 1994|Про Голову Верховної Ради України}}&lt;/ref&gt;. У червні 1994 р. [[Вибори Президента України 1994|балотувався на посаду президента України]], у першому турі здобув 13.04&amp;nbsp;% (3-тє місце серед 7 претендентів).

Зіграв одну з ключових ролей в ухваленні [[Конституція України|Конституції України]] (28 червня 1996 р.)

У 1998 потрапив до Верховної Ради за списком [[Виборчий блок Соціалістичної партії України та Селянської партії України «За правду, за народ, за Україну!» (1998)|блоку СПУ і СелПУ]], очолив фракцію «[[Лівий центр]]», до 2000 р. керував парламентським комітетом з питань аграрної політики та земельних відносин.

Був кандидатом у президенти від соціалістів на [[Вибори Президента України 1999|виборах 1999]] р. Разом з [[Ткаченко Олександр Миколайович (політик)|Олександром Ткаченком]], [[Марчук Євген Кирилович|Євгеном Марчуком]] та [[Олійник Володимир Миколайович|Володимиром Олійником]] створив [[Канівська четвірка|«канівську четвірку»]]. Але врешті-решт знімати свою кандидатуру на користь інших претендентів відмовився. У 1-му турі здобув 11.29&amp;nbsp;% (3-тє місце серед 13 кандидатів).

Восени 2000 р. оприлюднив «[[плівки Мельниченка]]», поклавши початок «[[касетний скандал|касетному скандалу]]». Активний учасник акції «[[Україна без Кучми]]». Член ради «&lt;nowiki/&gt;[[Форум національного порятунку|Форуму національного порятунку]]&lt;nowiki/&gt;» (у 2001). Представляв Громадянський комітет захисту Конституції на переговорах з представниками влади.

У 2002 р. обраний до парламенту за списком Соціалістичної партії, очолив фракцію [[Соціалістична партія України|СПУ]]. Працював в комітеті з питань правової політики.

Активно долучався до масштабних протестних акцій «[[Повстань, Україно!]]» у 2002—2003 рр.

У цьому парламенті разом зі [[Гавриш Степан Богданович|Степаном Гавришем]] очолював [[Тимчасова спеціальна комісія Верховної Ради України|ТСК]] по опрацюванню проектів законів України про внесення змін до Конституції України (2002).

У 2004 р. балотувався на [[Вибори Президента України 2004|виборах в Президенти України]]. В першому турі набрав 5,81&amp;nbsp;%. У другому турі підтримав [[Ющенко Віктор Андрійович|Віктора Ющенка]], поставивши однією з головних умов підтримки впровадження політичної реформи&amp;nbsp;— тобто перетворення України на [[Парламентсько-президентська республіка|парламентсько-президентську республіку]].

У 2006 році обраний до парламенту за списком [[Соціалістична партія України|Соціалістичної партії]], очолив фракцію. Незважаючи на підписану коаліційну угоду між фракціями [[БЮТ]], [[Блок «Наша Україна» (2006)|«Наша Україна»]] та [[Соціалістична партія України|СПУ]], яка передбачала обрання [[Юлія Тимошенко|Юлії Тимошенко]] прем'єр-міністром та [[Порошенко Петро Олексійович|Петра Порошенка]] спікером, несподівано висунув власну кандидатуру на посаду голови Верховної Ради, що було розцінено [[Блок «Наша Україна» (2006)|«Нашою Україною»]], [[БЮТ]] та частиною соціалістів, як зрада. БЮТ та НСНУ відмовилися брати участь у голосуванні, проте Мороза обрано&amp;nbsp;— завдяки підтримці фракцій [[Партія регіонів|ПРУ]] та [[Комуністична партія України (1993—2022)|КПУ]], які наступного дня разом із соціалістами утворили нову коаліцію більшості в парламенті&lt;ref&gt;{{Постанова ВРУ|7-V|6 липня 2006|Про Голову Верховної Ради України}}&lt;/ref&gt;.
[[Файл:Moroz Yushchenko.jpg|thumb|250px|Мороз (ліворуч) з Віктором Ющенком.]]
Обрання Мороза спікером ВРУ було суперечливе як із правового погляду, бо СПУ сформувала нову коаліцію не вийшовши із коаліції із БЮТ та НСНУ, а це є грубим порушенням регламенту ВР, так і з морально-етичного, бо це суперечило сподіванням значної кількості виборців СПУ. Наслідком цього було виникнення гострого протиборства між гілками влади за повноваження, ініційоване новоутвореною коаліцією,&amp;nbsp;— що згодом призвело до розпуску ВР президентом та початку дострокових виборів. Дані події також стали однією з головних причин кількаразового падіння рейтингів СПУ.

На [[Парламентські вибори в Україні 2007|виборах 2007 року]] Соціалістична партія не подолала 3-відсотковий бар'єр і Мороз не отримав депутатського мандата.

На [[Вибори Президента України 2010|президентських виборах 2010 року]] здобув підтримку 0,38&amp;nbsp;% виборців.

25 січня 2019 року рішенням [[Центральна виборча комісія України|ЦВК]] Олександра Мороза зареєстровано&lt;ref&gt;[https://www.cvk.gov.ua/pls/vp2019/wp005pt021f01=227pt001f01=719.html Кандидат на пост Президента України Мороз Олександр Олександрович] // Вибори Президента України 2019. Центральна виборча комісія України. 25.01.2019{{Webarchive|url=https://web.archive.org/web/20201102161644/https://www.cvk.gov.ua/pls/vp2019/wp005pt021f01=227pt001f01=719.html |date=2 листопада 2020 }}&lt;/ref&gt; кандидатом на посаду Президента України. Суб'єкт висування&amp;nbsp;— ПП «Соціалістична партія Олександра Мороза». 27 березня 2019 року Олександр Мороз зняв свою кандидатуру&lt;ref&gt;[https://www.5.ua/polityka/moroz-zniav-svoiu-kandydaturu-z-vyboriv-prezydenta-189264.html Мороз зняв свою кандидатуру з виборів Президента] // 5 канал. 27.03.2019. {{Webarchive|url=https://web.archive.org/web/20210524161541/https://www.5.ua/polityka/moroz-zniav-svoiu-kandydaturu-z-vyboriv-prezydenta-189264.html |date=24 травня 2021 }}&lt;/ref&gt;.

Сумарно балотувався на посаду президента України п'ять разів.

== Нагородження орденом князя Ярослава Мудрого ==
19 серпня 1998 року Президент України [[Кучма Леонід Данилович|Л.&amp;nbsp;Д.&amp;nbsp;Кучма]] видав указ №&amp;nbsp;899/98, яким «за видатні особисті заслуги перед Українською державою в галузі державного будівництва, значний внесок у розвиток законодавчої бази України та з нагоди 7-ї річниці незалежності України» постановив нагородити О.&amp;nbsp;О.&amp;nbsp;Мороза [[Орден князя Ярослава Мудрого|орденом князя Ярослава Мудрого]] V ст.&lt;ref&gt;Указ Президента України №&amp;nbsp;899/98 від 19 серпня 1998 року [http://zakon1.rada.gov.ua/cgi-bin/laws/main.cgi?nreg=899/98 «Про нагородження відзнакою Президента України „Орден князя Ярослава Мудрого“»]&lt;/ref&gt; О.&amp;nbsp;О.&amp;nbsp;Мороз від нагороди відмовився&lt;ref&gt;[http://www.spu.in.ua/kerivnitstvo/golova/10 Мороз Олександр Олександрович] {{Webarchive|url=https://web.archive.org/web/20100923103523/http://www.spu.in.ua/kerivnitstvo/golova/10 |date=23 вересня 2010 }} // Офіційний сайт СПУ&lt;/ref&gt;, і 11 березня 1999 року указом Л.&amp;nbsp;Д.&amp;nbsp;Кучми №&amp;nbsp;243/99 указ щодо нагородження №&amp;nbsp;899/98 був визнаний таким, що втратив чинність.&lt;ref&gt;Указ Президента України №&amp;nbsp;243/99 від 11 березня 1999 року [http://zakon1.rada.gov.ua/cgi-bin/laws/main.cgi?nreg=243/99 «Про визнання таким, що втратив чинність, Указу Президента України від 19 серпня 1998 року №&amp;nbsp;899».]&lt;/ref&gt;

== Творчість ==
Автор книжок:
{{стовпці|2}}
* «Куди йдемо?..» (1993),
* «Вибір», «Тема для роздумів» (1996),
* «Дорога, з якої не зійти», «Між вічними полюсами» (1999),
* «Хроніка одного злочину» (2000),
* «Про землю, Конституцію і не тільки» (2001),
* «Політична анатомія України» (2004),
* збірка поезій «…З відстані» (2004),
* збірки поезій «Обличчям до вогню», «Белый снег на каменной террасе» (2006),
* «До голови треба булави» (2016),
* «Долею оплачені слова» (2018),
* автор понад 2000 публікацій у періодиці.
&lt;/div&gt;

== Особисте життя ==
Одружений, має двох дочок — Ірину та Руслану.

== Хобі ==
[[Шахи]], [[поезія]].

== Примітки ==
{{reflist}}

== Посилання ==
* [http://leksika.com.ua/18860427/legal/moroz Мороз] // {{Юридична енциклопедія|3}} &lt;small&gt;{{Webarchive|url=https://web.archive.org/web/20161203101524/http://leksika.com.ua/18860427/legal/moroz |date=3 грудня 2016 }}&lt;/small&gt;
&lt;!--{{Вікіпосилання|Вікіцитати=Мороз Олександр Олександрович}}--&gt;
* [https://web.archive.org/web/20040221144259/http://www.spu.org.ua/ Офіційний сайт СПУ]
* [http://dovidka.com.ua/user/?code=460418 Довідник «Хто є хто в Україні», видавництво «К. І. С»] &lt;small&gt;{{Webarchive|url=https://web.archive.org/web/20201107113724/http://dovidka.com.ua/user/?code=460418 |date=7 листопада 2020 }}&lt;/small&gt;
* [http://ukrpes.net/vazhnie-sobytiya/moroza-izbrali-predsedatelem-spu.html Мороза обрали головою СПУ] &lt;small&gt;{{Webarchive|url=https://web.archive.org/web/20160304123238/http://ukrpes.net/vazhnie-sobytiya/moroza-izbrali-predsedatelem-spu.html |date=4 березня 2016 }}&lt;/small&gt; {{ref-ru}}

== Джерела ==
* ''Удод О. А.'' [http://history.org.ua/?encyclop&amp;termin=Moroz_O Мороз Олександр Олександрович] // {{ЕІУ|7|72}}

== Література ==
* ''Гарань О''. Мороз Олександр Олександрович // Політична енциклопедія / редкол.: Ю. Левенець (голова), Ю. Шаповал (заст. голови) та ін.&amp;nbsp;— К. : Парламентське видавництво, 2011.&amp;nbsp;— С. 467.&amp;nbsp;— ISBN 978-966-611-818-2.
* ''Губерський Л. В.'' Мороз Олександр Олександрович // [[Українська дипломатична енциклопедія]]: у 2-х т. / редкол.: Л.&amp;nbsp;В.&amp;nbsp;Губерський (голова) та ін.&amp;nbsp;— К. : Знання України, 2004.&amp;nbsp;— Т. 2.&amp;nbsp;— 812 с.&amp;nbsp;— ISBN 966-316-045-4.
{{1em}}
{{succession|office=[[Голови Верховної Ради України]] &lt;br/&gt; |preceded= [[Плющ Іван Степанович]]&lt;br/&gt; &lt;br/&gt; | succeeded= [[Яценюк Арсеній Петрович]]}}

{{Керівники українського парламенту}}
{{Кандидати у президенти України 2019}}
{{Кандидати у президенти України 2010}}
{{Кандидати у президенти України 2004}}
{{Кандидати у президенти України 1999}}
{{Кандидати у президенти України 1994}}
{{Золоті письменники України}}
{{ВП-портали|Київщина|Україна|Біографії}}
{{Бібліоінформація}}

[[Категорія:Уродженці Таращанського району]]
[[Категорія:Випускники Української сільськогосподарської академії]]
[[Категорія:Випускники Вищої партійної школи при ЦК КПУ]]
[[Категорія:Народні депутати України 1-го скликання]]
[[Категорія:Народні депутати України 2-го скликання]]
[[Категорія:Народні депутати України 3-го скликання]]
[[Категорія:Народні депутати України 4-го скликання]]
[[Категорія:Народні депутати України 5-го скликання]]
[[Категорія:Голови Верховної Ради України]]
[[Категорія:Члени РНБО]]
[[Категорія:Члени Соціалістичної партії України]]
[[Категорія:Учасники Помаранчевої революції]]
[[Категорія:Справа Гонгадзе]]
[[Категорія:Члени конституційних комісій в Україні]]
[[Категорія:Українські соціалісти]]
[[Категорія:Кандидати в президенти України (1994)]]
[[Категорія:Кандидати в президенти України (1999)]]
[[Категорія:Кандидати в президенти України (2004)]]
"""
p = wtp.parse(text)
print(p)


{{Інші значення|тип=прізвище|Мороз}}
{{Картка:Лідер
 | ім'я =Олександр Олександрович Мороз
 | зображення = Moroz Yushchenko cropped.jpg
 | розмір_зображення = 200px
 | підпис_зображення =
| посада =  7-й [[Голова Верховної Ради України]]
| початок_терміну = [[6 липня]] [[2006]]
| кінець_терміну =  [[23 листопада]] [[2007]]
| попередник = [[Литвин Володимир Михайлович|Володимир Литвин]]
| наступник = [[Яценюк Арсеній Петрович|Арсеній Яценюк]]
| президент = [[Ющенко Віктор Андрійович|Віктор Ющенко]]
| посада2 =  3-й [[Список Голів Верховної Ради України|Голова Верховної Ради України]]
| початок_терміну2 = [[18 травня]] [[1994]]
| кінець_терміну2 = [[12 травня]] [[1998]]
| попередник2 = [[Плющ Іван Степанович|Іван Плющ]]
| наступник2 = [[Ткаченко Олександр Миколайович (політик)|Олександр Ткаченко]]
| президент2 = [[Кравчук Леонід Макарович|Леонід Кравчук]] &lt;br/&gt; [[Кучма Леонід Данилович|Леонід Кучма]]
 &lt;!--| дата_народження = 29.2.1944&lt;ref name=&quot;ЕІУдата&quot; /&gt;[http:

In [92]:
i = 1
print(p.wikilinks[i])
print(p.wikilinks[i].title)
print(p.wikilinks[i].fragment)
print(p.wikilinks[i].text)
print(p.wikilinks[i].target)

[[Берлінське античне зібрання]]
Берлінське античне зібрання
None
None
Берлінське античне зібрання


In [112]:
i = 1
# print(p.sections[i])
print(p.sections[i].title.lower())
print(p.sections[i].level)

 життєпис 
2


In [20]:
i = 11
print(p.templates[i])
print(p.templates[i].normal_name())
for arg in p.templates[i].arguments:
    print('name =', arg.name, 'value =', arg.value)
# print(p.templates[i].normal_name())
# print(p.templates[i].get_arg('ім\'я'))
# p.templates[i].arguments[0].name.strip()

{{ДН|29|02|1944}}
ДН
name = 1 value = 29
name = 2 value = 02
name = 3 value = 1944


In [14]:
p.templates

[Template('{{Інші значення|тип=прізвище|Мороз}}'),
 Template("{{Картка:Лідер\n | ім'я =Олександр Олександрович Мороз\n | зображення = Moroz Yushchenko cropped.jpg\n | розмір_зображення = 200px\n | підпис_зображення =\n| посада =  7-й [[Голова Верховної Ради України]]\n| початок_терміну = [[6 липня]] [[2006]]\n| кінець_терміну =  [[23 листопада]] [[2007]]\n| попередник = [[Литвин Володимир Михайлович|Володимир Литвин]]\n| наступник = [[Яценюк Арсеній Петрович|Арсеній Яценюк]]\n| президент = [[Ющенко Віктор Андрійович|Віктор Ющенко]]\n| посада2 =  3-й [[Список Голів Верховної Ради України|Голова Верховної Ради України]]\n| початок_терміну2 = [[18 травня]] [[1994]]\n| кінець_терміну2 = [[12 травня]] [[1998]]\n| попередник2 = [[Плющ Іван Степанович|Іван Плющ]]\n| наступник2 = [[Ткаченко Олександр Миколайович (політик)|Олександр Ткаченко]]\n| президент2 = [[Кравчук Леонід Макарович|Леонід Кравчук]] &lt;br/&gt; [[Кучма Леонід Данилович|Леонід Кучма]]\n &lt;!--| дата_народження = 29.2.1944&lt

In [169]:
_filter_page_text(text, log=True)

matched template arg "лідер_ім'я" with "ім'я"


True

In [19]:
rx = re.compile(r'\d{1,2}\.\d{1,2}\.\d{4}')
t = '| Birth date     = 12.4.1867'
m = rx.search(t)

In [27]:
m.group()

'12.4.1867'